### Imports

In [1]:
from breeze_connect import BreezeConnect
from jproperties import Properties
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from pandas_market_calendars import get_calendar
import opstrat as op

### Utility Methods

In [2]:
def get_number_of_trading_sessions(start_date, end_date):
    # Get the NSE calendar
    nse = get_calendar('NSE')

    # Get the trading sessions between the start and end dates
    trading_sessions = nse.schedule(start_date=start_date, end_date=end_date)

    # Return the number of trading sessions
    return len(trading_sessions)

assert(get_number_of_trading_sessions('2023-06-26', '2023-07-02') == 4)



In [3]:
getDF = lambda data : pd.DataFrame(data["Success"])

In [ ]:
# Example usage
start_date = '2023-06-30'
end_date = '2023-09-28'
num_trading_sessions = get_number_of_trading_sessions(start_date, end_date)
print("Number of trading sessions:", num_trading_sessions)

### Login to Breeze API

In [4]:
configs = Properties()
with open('app.properties', 'rb') as read_prop:
    configs.load(read_prop,"utf-8")
if len(configs) == 0:
    print("Error while reading properties files")

api_key = configs.get("APP_KEY").data if configs.get("APP_KEY") else None
api_secret = configs.get("SECRET_KEY").data if configs.get("SECRET_KEY") else None
session_token =configs.get("SESSION_TOKEN").data if configs.get("SESSION_TOKEN") else None
breeze = BreezeConnect(api_key=api_key)
breeze.generate_session(api_secret=api_secret,session_token=session_token)

### get_portfolio_holdings

In [5]:
getDF(breeze.get_portfolio_holdings(exchange_code="NSE",
                                from_date="2022-08-01T06:00:00.000Z",
                                to_date="2022-08-19T06:00:00.000Z",
                                stock_code="",
                                portfolio_type=""))

,stock_code,exchange_code,quantity,average_price,booked_profit_loss,current_market_price,change_percentage,answer_flag,product_type,expiry_date,strike_price,right,category_index_per_stock,action,realized_profit,unrealized_profit,open_position_value,portfolio_charges
0,NIFBEE,NSE,5,193.92,0,213.96,0.488446364831862,N,None,None,None,None,None,None,None,None,None,None
1,INDPAI,NSE,3,1487.54,0,1449.25,1.4703308244355,N,None,None,None,None,None,None,None,None,None,None


### Historic Data

In [84]:
# For equity
interval = "1day" # "1minute", "5minute", "30minute" or as "1day"
from_date = "17-Jun-2013"
to_date = "06-Jul-2023"
stock_code = "NIFTY"
exchange_code = "NSE"
product_type = "cash"


# get historical data for option premium for a particular strike price and underlying data
df = getDF(breeze.get_historical_data(interval,from_date,to_date,stock_code,exchange_code,product_type))

colsToDrop = ['stock_code', 'product_type', 'exchange_code','expiry_date','right','strike_price','count','open_interest',]
df =df.drop(columns =colsToDrop)
df['datetime'] = pd.to_datetime(df['datetime']).dt.normalize()
colsToFloat = ['open','close','high','low']
df[colsToFloat] = df[colsToFloat].apply(pd.to_numeric)
df['hc_diff'] = df['high'] - df['low']
df['oc_diff'] = df['open'] - df['close']
len(df)

2469

In [85]:
df.sort_values(by='datetime')
for i in range(1, len(df)):
    df.loc[i,'ocPrev_diff'] = df.loc[i, 'open'] -df.loc[i-1, 'close']
df = df.reindex(df.ocPrev_diff.abs().sort_values(ascending=False).index)
df.head(255)


,datetime,open,high,low,close,volume,hc_diff,oc_diff,ocPrev_diff
1653,2020-03-23,7945.70,8159.25,7583.60,7610.25,1940141559,575.65,335.45,-799.75
2133,2022-02-24,16548.90,16705.25,16203.25,16247.95,385954350,502.00,300.95,-514.35
1647,2020-03-13,9107.60,10159.40,8555.15,9955.20,450017974,1604.25,-847.60,-482.55
824,2016-11-09,8067.50,8476.20,8002.25,8432.00,1696067110,473.95,-364.50,-476.05
1646,2020-03-12,10039.95,10040.75,9508.00,9590.15,378117001,532.75,449.80,-418.45
...,...,...,...,...,...,...,...,...,...
1292,2018-10-04,10754.70,10754.70,10547.25,10599.25,1681161596,207.45,155.45,-103.55
1816,2020-11-14,12823.35,12828.70,12749.45,12780.25,791508963,79.25,43.10,103.40
2122,2022-02-09,17370.10,17477.15,17339.00,17463.80,229829643,138.15,-93.70,103.35
1663,2020-04-08,8688.90,9131.70,8653.90,8748.75,245044864,477.80,-59.85,-103.30


In [86]:
len(df[df.ocPrev_diff.abs() >400])

7

### write to csv

In [50]:
df.to_csv('file1.csv')

In [15]:
# For options
interval = "30minute" # "1minute", "5minute", "30minute" or as "1day"
from_date = "01-Apr-2023"
to_date = "28-Jun-2023"
stock_code = "NIFTY"
exchange_code = "NFO"
product_type = "options"
expiry_date="28-Jun-2023"
right="call"
strike_price="17000"
columnsToDrop = ['stock_code', 'product_type', 'exchange_code','expiry_date','right','strike_price','volume','open_interest',]

# get historical data for option premium for a particular strike price and underlying data
df_call = getDF(breeze.get_historical_data(interval,from_date,to_date,stock_code,exchange_code,product_type,expiry_date,right,strike_price))
# df =df.drop(columns =columnsToDrop)

df_put =  getDF(breeze.get_historical_data(interval,from_date,to_date,stock_code,exchange_code,product_type,expiry_date,"put",strike_price))

In [16]:
df_put

,datetime,stock_code,exchange_code,product_type,expiry_date,right,strike_price,open,high,low,close,volume,open_interest,count
0,2023-04-03 09:15:00,NIFTY,NFO,Options,28-JUN-23,Put,17000,221.25,257,221.25,236.15,47400,1514700,0
1,2023-04-03 09:45:00,NIFTY,NFO,Options,28-JUN-23,Put,17000,237.1,238.05,230.2,232.15,20450,1520750,1
2,2023-04-03 10:15:00,NIFTY,NFO,Options,28-JUN-23,Put,17000,233.15,241.9,230.9,240.3,8550,1521550,2
3,2023-04-03 10:45:00,NIFTY,NFO,Options,28-JUN-23,Put,17000,242,245,237.45,245,9050,1524850,3
4,2023-04-03 11:15:00,NIFTY,NFO,Options,28-JUN-23,Put,17000,244.8,247.3,240.1,240.1,6700,1526250,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,2023-06-28 13:15:00,NIFTY,NFO,Options,28-JUN-23,Put,17000,.75,.85,.65,.75,649900,5073850,762
763,2023-06-28 13:45:00,NIFTY,NFO,Options,28-JUN-23,Put,17000,.75,.85,.7,.8,599850,4980550,763
764,2023-06-28 14:15:00,NIFTY,NFO,Options,28-JUN-23,Put,17000,.8,.8,.25,.3,754500,4896500,764
765,2023-06-28 14:45:00,NIFTY,NFO,Options,28-JUN-23,Put,17000,.3,.3,.1,.15,759100,4854200,765


### get_option_chain_quotes

In [91]:
# For stock
t =breeze.get_option_chain_quotes(stock_code="ITC",
                    exchange_code="NFO",
                    product_type="options",
                    strike_price="400",
                    right="call")
getDF(t)

,exchange_code,product_type,stock_code,expiry_date,right,strike_price,ltp,ltt,best_bid_price,best_bid_quantity,...,ltp_percent_change,upper_circuit,lower_circuit,total_quantity_traded,spot_price,ltq,open_interest,chnge_oi,total_buy_qty,total_sell_qty
0,NFO,Options,ITC,27-Jul-2023,Call,400.0,75.70,06-Jul-2023 15:14:49,75.40,1600,...,-1.11,101.50,52.10,9600,473.9,1600,1137600.0,-8000.0,68800,67200
1,NFO,Options,ITC,31-Aug-2023,Call,400.0,77.55,06-Jul-2023 15:14:34,74.25,1600,...,26.82,104.15,56.05,1600,473.9,1600,1600.0,0.0,65600,65600
2,NFO,Options,ITC,28-Sep-2023,Call,400.0,0.00,,73.70,1600,...,-100.00,106.50,59.50,0,473.9,0,0.0,0.0,1600,1600


In [125]:
def extract(t,colsToDrop,low=None,high=None):
    df =getDF(t)
    df=df[df['ltp'] > 0] 
    df=df[df['strike_price'] > low] if low else df
    df=df[df['strike_price'] < high] if high else df
    df =df.drop(columns =colsToDrop)
    return df

def get_opt_chn(right,low=None,high=None):
    t=breeze.get_option_chain_quotes(stock_code,exchange_code,product_type=product_type,expiry_date=expiry_date,right=right)
    return extract(t,colsToDrop,low,high)

In [127]:
# for index
stock_code = "NIFTY"
exchange_code = "NFO"
product_type = "options"
expiry_date = "28-Sep-2023" # put future expiry date
colsToDrop = ['stock_code', 'product_type', 'exchange_code']
low=17700
high=19900


df_call = get_opt_chn("call")
df_put = get_opt_chn("put")
df_op_chain= df_call[['expiry_date','strike_price','ltp']].merge(df_put[['strike_price','ltp']], on = 'strike_price')
df_op_chain=df_op_chain.rename(columns={"ltp_x": "call", "ltp_y": "put","strike_price":"strike","expiry_date":"expiry"})
df_op_chain['diff_premium'] =abs(df_op_chain['call'] - df_op_chain['put'])
df_op_chain['sum'] =df_op_chain['call'] + df_op_chain['put']
df_op_chain = df_op_chain.reindex(df_op_chain.diff_premium.sort_values().index)
type(df_op_chain)
df_op_chain.head(10)

,expiry,strike,call,put,diff_premium,sum
32,28-Sep-2023,19750.0,405.00,426.00,21.00,831.00
31,28-Sep-2023,19700.0,429.40,406.60,22.80,836.00
33,28-Sep-2023,19800.0,374.85,455.95,81.10,830.80
30,28-Sep-2023,19650.0,470.00,388.50,81.50,858.50
29,28-Sep-2023,19600.0,486.90,368.20,118.70,855.10
34,28-Sep-2023,19850.0,347.05,478.10,131.05,825.15
28,28-Sep-2023,19550.0,504.80,348.95,155.85,853.75
35,28-Sep-2023,19900.0,326.20,502.10,175.90,828.30
27,28-Sep-2023,19500.0,547.30,332.60,214.70,879.90
26,28-Sep-2023,19450.0,575.40,315.10,260.30,890.50


In [130]:
df=df[df['ltp'] > 0] 
df['spot_price'] = df['spot_price'].astype('float')
# df['intrinsic_value'] = np.maximum(0,df['spot_price'] - df['strike_price'])
# df=df[df['intrinsic_value'] > 0] 
#df=df[df['strike_price'] > 18000] 
df['intrinsic_value'] =df['spot_price'] - df['strike_price']
df['time_value'] = df['ltp'] - df['intrinsic_value']
df[['spot_price','strike_price','ltp','intrinsic_value','time_value']]

KeyError: 'strike'

### get_quotes

In [88]:
stock_code = "NIFTY"
exchange_code = "NSE"
t = breeze.get_quotes(stock_code=stock_code,
                    exchange_code=exchange_code)
df = getDF(t)
df=df[df['exchange_code'] == exchange_code]
df

,exchange_code,product_type,stock_code,expiry_date,right,strike_price,ltp,ltt,best_bid_price,best_bid_quantity,...,best_offer_quantity,open,high,low,previous_close,ltp_percent_change,upper_circuit,lower_circuit,total_quantity_traded,spot_price
0,NSE,,NIFTY,None,None,0.0,19497.3,06-Jul-2023 16:21:40,0.0,0,...,0,19385.7,19512.2,19373.0,19398.5,0.509318,0.0,0.0,0,None


In [89]:
columnsToDrop = ['exchange_code', 'expiry_date', 'right','product_type','strike_price']
df1 =df.drop(columns =columnsToDrop)
df1

,stock_code,ltp,ltt,best_bid_price,best_bid_quantity,best_offer_price,best_offer_quantity,open,high,low,previous_close,ltp_percent_change,upper_circuit,lower_circuit,total_quantity_traded,spot_price
0,NIFTY,19497.3,06-Jul-2023 16:21:40,0.0,0,0.0,0,19385.7,19512.2,19373.0,19398.5,0.509318,0.0,0.0,0,None


In [90]:
# bar plot
df.plot(kind='line',
        x='ltp',
        y='spot_price',
        color='green')
 

TypeError: no numeric data to plot